# 构造微调训练数据集

借助 deepseek-chat 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 deepseek-chat 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-4o-mini` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 deepseek-chat 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-4o-mini 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-4o-mini&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  max_tokens=4095,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

---

## 使用 LangChain 构造训练数据

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
import httpx

# 1. 创建异步客户端（AsyncClient）
async_client = httpx.AsyncClient()  # 不需要代理则去掉 proxies
http_client = httpx.Client()

chat = ChatOpenAI(model="deepseek-chat",
                  base_url="https://api.deepseek.com",
                  http_client=http_client,
                  async_client=async_client,  # 关键点：用 http_client 代替 proxies
                  temperature=1,
                  max_tokens=4095)

/root/miniconda3/envs/peft/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [3]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [4]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [5]:
ai_message = chat(messages)

/root/miniconda3/envs/peft/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


### 解析 OpenAI GPT 生成的训练数据

In [6]:
ai_message.content

'content:"蒙卦"\nsummary:"蒙卦在周易中象征着启蒙与教育的智慧，由下卦坎（水）和上卦艮（山）组成。这一卦象描绘了山下涌出泉水的景象，寓意着知识如同初出山涧的泉水，虽细小却终将汇聚成江河。正如蒙昧无知的状态需要逐步开启，教育在此过程中扮演着至关重要的角色。\n\n蒙卦的卦象中，艮（山）代表止，坎（水）代表险，形成\'山下有险\'的格局。这象征着在事物发展的初期阶段，人们常因面临险阻而停滞不前，处于蒙昧不明的状态。然而，正是这种蒙昧促使教育成为当务之急，培养纯正无邪的品质被视为开启蒙昧的根本之道。\n\n尽管蒙卦呈现了初始阶段的蒙昧状态，但它同时强调了把握时机的重要性。适时而恰当的行动能够带来启蒙与通达，这正是蒙卦所蕴含的积极意义。\n\n在卦序中，蒙卦紧随屯卦（始生卦）之后，《序卦》解释道：\'物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。\'这揭示了万物在初始阶段都必然经历蒙昧时期，如同人类的童年阶段需要启蒙教育。\n\n《象传》对蒙卦的解读是：\'山下出泉，蒙；君子以果行育德。\'这提醒君子应当以果断的行动来培育德行，体现了启蒙教育中知行合一的重要理念。"'

In [7]:
text = ai_message.content

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 蒙卦
Summary: 蒙卦在周易中象征着启蒙与教育的智慧，由下卦坎（水）和上卦艮（山）组成。这一卦象描绘了山下涌出泉水的景象，寓意着知识如同初出山涧的泉水，虽细小却终将汇聚成江河。正如蒙昧无知的状态需要逐步开启，教育在此过程中扮演着至关重要的角色。

蒙卦的卦象中，艮（山）代表止，坎（水）代表险，形成'山下有险'的格局。这象征着在事物发展的初期阶段，人们常因面临险阻而停滞不前，处于蒙昧不明的状态。然而，正是这种蒙昧促使教育成为当务之急，培养纯正无邪的品质被视为开启蒙昧的根本之道。

尽管蒙卦呈现了初始阶段的蒙昧状态，但它同时强调了把握时机的重要性。适时而恰当的行动能够带来启蒙与通达，这正是蒙卦所蕴含的积极意义。

在卦序中，蒙卦紧随屯卦（始生卦）之后，《序卦》解释道：'物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。'这揭示了万物在初始阶段都必然经历蒙昧时期，如同人类的童年阶段需要启蒙教育。

《象传》对蒙卦的解读是：'山下出泉，蒙；君子以果行育德。'这提醒君子应当以果断的行动来培育德行，体现了启蒙教育中知行合一的重要理念。


### 持久化存储训练数据集文件

In [19]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [8]:
def generate_question_summary_pairs(content, summary):
    """
    生成25对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？",
        "{}的基本含义是什么？",
        "如何用现代语言简单解释{}？",
        "{}象征什么自然现象或事物？",
        "{}的想传达什么根本信息？",
        "周易中{}揭示了什么根本规律？",
        "周易中是如何定义{}的？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

In [9]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [10]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [11]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国

### 将以上的所有模块，整合到一起，自动化生成数据

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

import httpx

# 1. 创建异步客户端（AsyncClient）
async_client = httpx.AsyncClient()  # 不需要代理则去掉 proxies
http_client = httpx.Client()

# 初始化LangChain的GPT-4o-mini调用
chat = ChatOpenAI(model="deepseek-chat",
                  base_url="https://api.deepseek.com",
                  http_client=http_client,
                  async_client=async_client,  # 关键点：用 http_client 代替 proxies
                  temperature=1,
                  max_tokens=4095)

def gen_data(raw_content):
    """
    使用 deepseek-chat 调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: deepseek-chat 模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [13]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

content:"蒙卦"
summary:"在周易哲学体系中，蒙卦（艮上坎下）蕴含着深刻的启蒙智慧。卦象上艮为山，下坎为水，构成'山下出泉'的意象，象征着智慧如泉水般从蒙昧中涌出。卦辞'亨。匪我求童蒙，童蒙求我'揭示了教育的基本法则：真正的启蒙应当源于学习者主动求教，而非教育者强行灌输。

蒙卦的核心哲学体现在'初筮告，再三渎'的警示中，强调学习需要虔诚专注的态度。第一次求问能得到启示，但若反复轻慢地占问，就会失去获得智慧的机缘。这一原理同样适用于现代教育——只有保持谦逊求教的心态，才能获得真知。

《象传》'君子以果行育德'指出，面对蒙昧状态，君子应当像山泉般果敢前行，在行动中培养德行。北宋邵雍解读此卦'智慧未开，蒙昧闭塞'，傅佩荣则强调'蓄积德行'的重要性。蒙卦虽主'疑惑不前'的凶象，但通过接受良师教诲、培养果断品格，可转凶为吉。

在人生应用层面，蒙卦启示我们：
1. 事业初创期需忍耐待机，多听取建议
2. 经商要树立道德信誉，忌急功近利
3. 求名须打好基础，保持纯正动机
4. 婚恋要重品德考察，夫妻相互包容
5. 决策时要克服犹豫，接受历练考验

蒙卦最终传达的是'启蒙通达'的智慧：承认自身的蒙昧，以开放心态求教，通过持续修行终将拨云见日。"


In [14]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [15]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 蒙卦
Summary: 在周易哲学体系中，蒙卦（艮上坎下）蕴含着深刻的启蒙智慧。卦象上艮为山，下坎为水，构成'山下出泉'的意象，象征着智慧如泉水般从蒙昧中涌出。卦辞'亨。匪我求童蒙，童蒙求我'揭示了教育的基本法则：真正的启蒙应当源于学习者主动求教，而非教育者强行灌输。

蒙卦的核心哲学体现在'初筮告，再三渎'的警示中，强调学习需要虔诚专注的态度。第一次求问能得到启示，但若反复轻慢地占问，就会失去获得智慧的机缘。这一原理同样适用于现代教育——只有保持谦逊求教的心态，才能获得真知。

《象传》'君子以果行育德'指出，面对蒙昧状态，君子应当像山泉般果敢前行，在行动中培养德行。北宋邵雍解读此卦'智慧未开，蒙昧闭塞'，傅佩荣则强调'蓄积德行'的重要性。蒙卦虽主'疑惑不前'的凶象，但通过接受良师教诲、培养果断品格，可转凶为吉。

在人生应用层面，蒙卦启示我们：
1. 事业初创期需忍耐待机，多听取建议
2. 经商要树立道德信誉，忌急功近利
3. 求名须打好基础，保持纯正动机
4. 婚恋要重品德考察，夫妻相互包容
5. 决策时要克服犹豫，接受历练考验

蒙卦最终传达的是'启蒙通达'的智慧：承认自身的蒙昧，以开放心态求教，通过持续修行终将拨云见日。


In [16]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)


In [17]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦是一个富有启示意义的卦象，由下卦坎（水）和上卦艮（山）组成。这一卦象象征着蒙昧与启蒙，如同山下涌出的泉水，代表着初始的混沌与逐渐明晰的过程。卦辞“亨。匪我求童蒙，童蒙求我”表明，启蒙不是主动施与，而是需要求知者主动寻求。第一次占筮会得到神灵的启示，但若轻慢不敬地再三占问，则不会得到回应。

蒙卦的核心哲学在于：人在蒙昧时期需要接受教育和引导，以开启智慧。君子观此卦象，应当效仿山泉果敢前行的精神，以坚定的行动培育品德。此卦虽主蒙昧和疑惑，但若能顺应贤师良友的教诲，便可转迷为悟，通达顺利。

《象辞》解释为“山下出泉，蒙”，强调君子需以果敢之行育德。从运势来看，蒙卦提示初始阶段的迷惑与危机，需耐心待机，听取他人意见。在事业上，混乱中需以勇敢行动扭转局面，同时接受严格教育，脚踏实地。经商则需谨慎，树立高尚道德。求名者需纯正动机，婚恋需注重品德考察，决策时需克服疑惧，接受考验，等待时机。
Content: 屯卦"

summary:"在周易哲学体系中，屯卦（震下坎上）象征着万物初生时的艰难状态。卦辞'元亨利贞'昭示着事物发展的根本规律，而'勿用有攸往'则告诫此时不宜妄动。这个卦象生动展现了云雷交作的天地初开之象，君子观此卦象，当领悟'经纶'之道——既要如云施恩泽，又要似雷行威严。

从卦象结构来看，下卦震（雷）代表萌动之力，上卦坎（水）象征险陷之境，构成'动乎险中'的深刻意境。邵雍将其阐释为'万物始生，开始困难'的生命哲理，傅佩荣则从现实角度指出'宜守不宜进'的处世智慧。传统解卦强调，虽然'雷雨交加，险象丛生'，但若能'顺时应运'，终将'欣欣向荣'。

在人生实践中，屯卦启示我们：
1. 事业初创期要'知难而进'又'小心翼翼'
2. 经商之道在于'坚定信念'与'果断行动'的平衡
3. 婚恋关系需要'忠贞纯洁'与'大胆追求'的结合
4. 决策之时当具'坚忍不拔'的毅力

值得注意的是，屯卦特别强调'时运'的重要性，指出困境中既要积极进取，又要懂得退守待时。这种辩证思维体现了周易'变易不易'的深层智慧，对现代人应对人生挑战具有重要启示意义。
Summary: 在周易哲学体系中，屯卦（震下坎上）象征着万物初生时的艰难状态。卦辞'元亨利贞'昭示着事物发展的根本规律，而'勿用有攸往'则告诫此时不宜妄动。这个卦象生动展现了云雷交作

### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 gpt-4o-mini 生成结果发现问题

In [18]:
def gen_data(raw_content):
    """
    使用deepseek-chat调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: deepseek-chat模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [19]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦是一个富有象征意义的卦象，由下卦坎（水）和上卦艮（山）组成。卦象为山下有泉，象征着启蒙和教育的初始阶段。蒙卦的核心含义是“启蒙”，强调学习者和教育者之间的关系：不是教育者主动去寻求学习者，而是学习者应当主动寻求知识和指导。

蒙卦的卦辞指出，初次求教时会得到真诚的回应，但若反复轻慢地询问，则会失去这种机会。这反映了对知识和教育的尊重态度。同时，卦辞也表明，保持正直和耐心是吉利的关键。

从哲学角度看，蒙卦象征着人在蒙昧无知的状态下，需要通过正确的引导和自我修养来开启智慧。君子观此卦象，应效仿山泉果决前行的精神，以坚定的行动培养品德。

在《断易天机》中，蒙卦被视为凶卦，主疑惑和犹豫；而邵雍则认为此卦代表智慧未开，需要贤师指引。傅佩荣则从时运、财运、家宅等多方面解读，强调德行积累和果决行动的重要性。

传统解卦指出，蒙卦虽象征初始的混乱和危机，但通过勇敢坚毅的行动和接受教育，可以扭转局面。在事业、经商、求名、婚恋等方面，蒙卦都强调脚踏实地、动机纯正和品德考察的重要性。决策时，需克服疑惧，接受考验，等待适当时机。
Content: 屯卦
Summary: 在周易中，屯卦象征着万物初生时的艰难与希望。它由下卦震（雷）和上卦坎（水）组成，形成云雷交加的景象。卦辞“元，亨，利，贞”预示大吉大利，但“勿用，有攸往”又提示暂时不宜远行，而“利建侯”则表明此时适合奠定基业。

从卦象看，云行雷动展现出生机与险阻并存的状态。君子观此卦象，领悟到治理之道应当效法云的恩泽与雷的威严，刚柔并济。《断易天机》指出此卦代表动中遇险，需要刚毅果敢才能化险为夷。

邵雍的解读强调万物始生必经历艰难，唯有坚持才能苦尽甘来。得此卦者正处于人生或事业的初创阶段，需知守成不易，应当保持耐心，脚踏实地克服困难。傅佩荣则从现实角度指出，此时运势宜守不宜进，创业维艰，需要保存实力。

传统解卦进一步阐释：震为动，坎为险，犹如雷雨交加时的艰难环境。但正如植物破土而出，只要顺应时势，终将迎来蓬勃发展。这个卦象提示我们，面对困境时要保持坚定信念，既要有勇往直前的决心，又要懂得灵活应变。在事业、经商、求名等方面，都需要经历初期的磨砺，方能成就大事。婚恋方面则预示经过考验的感情更为牢固。

决策时需注意：虽然前途光明，但初期孤独困苦在所难免。若能保持乐观，并得贤者相